<a href="https://colab.research.google.com/github/liriacs/analise_tecnica/blob/main/AT_efic_candles_engolf_harami_janela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance 
import yfinance as yf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.dates as mdates

In [ ]:
#esta biblioteca vem do video de investimento com ia e python - medias
#!pip install mpl_finance
#from mpl_finance import candlestick_ohlc

In [ ]:
'''baixar dados desde dez/2020 para que seja possivel calcular a media desde o inicio de janeiro de 2021'''
data = yf.download("petr4.SA", start="2019-12-01", end="2022-01-15")
#data

[*********************100%***********************]  1 of 1 completed


In [ ]:
data_teste = data
data_aux = data.copy()


In [ ]:
#obv

obv = []
obv_aux = data_teste['Volume'][0]
obv.append(obv_aux)
for i in range(1,len(data_teste)):
  #print(data_teste.index[i])
  if (data_teste['Close'][i] > data_teste['Close'][i-1]):
    obv_aux = obv_aux + data_teste['Volume'][i]
  elif (data_teste['Close'][i] < data_teste['Close'][i-1]):
    obv_aux = obv_aux - data_teste['Volume'][i]
  obv.append(obv_aux)
data_teste['obv_teste'] = obv
data_aux['obv_teste'] = obv

#data_teste.head()


In [ ]:
#medias
data_teste["MME8"] = data_teste["Close"].ewm(span=8, min_periods=8).mean()
data_teste["MME20"] = data_teste["Close"].ewm(span=20, min_periods=20).mean()
#data_teste["MME20vol"] = data_teste["Volume"].ewm(span=20, min_periods=20).mean()
#data_teste["MME8vol"] = data_teste["Volume"].ewm(span=8, min_periods=8).mean()
#data_teste["MME8obv"] = data_teste["OBV Change"].ewm(span=8, min_periods=8).mean()
#data_teste["MME20obv"] = data_teste["OBV Change"].ewm(span=20, min_periods=20).mean()

In [ ]:
#tentando identificar topo no obv
topos_obv = []
sentido = []
penult_topo = data_teste["obv_teste"][0]
ultimo_topo = data_teste["obv_teste"][0]
sentido_aux = ''
for i in range(len(data_teste)-1):
  #sentido_aux = ''
  if data_teste["obv_teste"][i-1] < data_teste["obv_teste"][i] > data_teste["obv_teste"][i+1]:
    penult_topo = ultimo_topo
    ultimo_topo = data_teste["obv_teste"][i]
    if penult_topo < ultimo_topo:
      sentido_aux = 'ASC'
    elif penult_topo > ultimo_topo:
      sentido_aux = 'DESC'
    elif penult_topo == ultimo_topo:
      sentido_aux = 'LAT'
  topos_obv.append(ultimo_topo)
  sentido.append(sentido_aux)
  #print(data_teste.index[i], ultimo_topo)

for i in range(len(data_teste)-1, len(data_teste)):
  topos_obv.append(ultimo_topo)
  sentido.append(sentido_aux)

#data_teste['topos_obv'] = topos_obv
data_teste['sentido_obv'] = sentido
data_aux['sentido_obv'] = sentido

In [ ]:
#Identificando topo do preço.
topos_preco = []
sentido_preco = []
penult_topo_pr = max([data_teste['Close'][0], data_teste['Open'][0]])
ultimo_topo_pr = max([data_teste['Close'][0], data_teste['Open'][0]])
sentido_aux_pr = ''
for i in range(len(data_teste)-1):
  #sentido_aux_pr = ''
  #if data_teste["Close"][i-2] < data_teste["Close"][i-1]  < data_teste["Close"][i] > data_teste["Close"][i+1]:
  consid0 = max([data_teste['Close'][i-2], data_teste['Open'][i-2]])
  consid1 = max([data_teste['Close'][i-1], data_teste['Open'][i-1]])
  consid2 = max([data_teste['Close'][i], data_teste['Open'][i]])
  consid3 = max([data_teste['Close'][i+1], data_teste['Open'][i+1]])
  

  #e se usar o valor maximo?
  '''
  # tem ponto que nao vai dar certo, pq o maximo do candle i+1 é maior. entretanto, é um topo
  consid0 = data_teste['High'][i-2]
  consid1 = data_teste['High'][i-1]
  consid2 = data_teste['High'][i]
  consid3 = data_teste['High'][i+1]
  '''

  '''
  # e se usar o maximo só em i?
  #tbm nao ficou mto bom. qdo tem varios candles formando um topo, acaba nao identificando
  consid0 = max([data_teste['Close'][i-2], data_teste['Open'][i-2]])
  consid1 = max([data_teste['Close'][i-1], data_teste['Open'][i-1]])
  consid2 = data_teste['High'][i]
  consid3 = max([data_teste['Close'][i+1], data_teste['Open'][i+1]])
  '''

  if consid0 < consid1  < consid2 >= consid3:
    penult_topo_pr = ultimo_topo_pr
    ultimo_topo_pr = consid2 #max(abertura, fechamento)[i]
    variacao = ultimo_topo_pr / penult_topo_pr
    
    '''if penult_topo_pr < ultimo_topo_pr:
      sentido_aux_pr = 'ASC'
    elif penult_topo_pr > ultimo_topo_pr:
      sentido_aux_pr = 'DESC'
    elif penult_topo_pr == ultimo_topo_pr:
      sentido_aux_pr = 'LAT'
      '''
    #if 0.9895 < variacao < 1.0105:
    if 0.9979 < variacao < 1.0021:
      sentido_aux_pr = 'LAT'
    elif penult_topo_pr < ultimo_topo_pr and variacao > 1.0021:
      sentido_aux_pr = 'ASC'
    elif penult_topo_pr > ultimo_topo_pr and variacao < 0.9979:
      sentido_aux_pr = 'DESC'
    #print('data ', data_teste.index[i], 'penult ', penult_topo_pr, ' ultimo ', ultimo_topo_pr, ' sentido ', sentido_aux_pr)
  topos_preco.append(ultimo_topo_pr)
  sentido_preco.append(sentido_aux_pr)
  #print(data_teste.index[i], ultimo_topo)
  #if 25 < i <58:
    #print(i, data_teste.index[i], 'penultimo ', penult_topo_pr, ' ultimo ', ultimo_topo_pr, 'var ' , variacao)

for i in range(len(data_teste)-1, len(data_teste)):
  topos_preco.append(ultimo_topo_pr)
  sentido_preco.append(sentido_aux_pr)

data_teste['topos_preco'] = topos_preco
data_teste['sentido_topos_preco'] = sentido_preco

data_aux['topos_preco'] = topos_preco
data_aux['sentido_topos_preco'] = sentido_preco

In [ ]:
#funcoes que buscam os padroes de candles
def func_j_alta(x):
  if (data_teste['Low'][x] > data_teste['High'][x-1] + 0.3 ):
    j_alta.append('sim')
  else:
    j_alta.append('não')

def func_j_baixa(x):
  if (data_teste['High'][x] < data_teste['Low'][x-1] + 0.3 ):
    #print('gap baixa')
    J_baixa.append('sim')
  else:
    #print('não')
    J_baixa.append('não')

def func_eng_alta(i):
  if(
    #PF1<PA1
    (data_teste['Close'][i-1] < data_teste['Open'][i-1])
    and 
    # PF2>PA2
    (data_teste['Close'][i] > data_teste['Open'][i])
    and
    #PF2>PA1
    (data_teste['Close'][i] > data_teste['Open'][i-1])
    and
    #PA2<PF1
    (data_teste['Open'][i]  < data_teste['Close'][i-1])
    and 
    #(PF2-PA2)/(PA1-PF1)>2
     ((data_teste['Close'][i] - data_teste['Open'][i] ) / (data_teste['Open'][i-1] - data_teste['Close'][i-1]) > 2)
    and 
    #(MA2-ME2)>(MA1-ME1)
    ((data_teste['High'][i]  - data_teste['Low'][i]) > ( data_teste['High'][i-1]  - data_teste['Low'][i-1] ))
  ):
    #print(data_teste.index[i], 'sim')
    Eng_alta.append('sim')
  else:
    #print(data_teste.index[i], 'nao')
    Eng_alta.append('não')

def func_eng_baixa(i):
  if(
     #PF1>PA1
     data_teste['Close'][i-1] > data_teste['Open'][i-1]
     and
     #PF2<PA2
     data_teste['Close'][i] < data_teste['Open'][i] 
     and
     #PF2<PA1
     data_teste['Close'][i] < data_teste['Open'][i-1]
     and
     #PA2>PF1
     data_teste['Open'][i]  > data_teste['Close'][i-1]
     and
     #(PA2-PF2)/(PF1-PA1)>2
     ((data_teste['Open'][i]  - data_teste['Close'][i]) / (data_teste['Close'][i-1] - data_teste['Open'][i-1] ) ) > 2
     and
     #(MA2-ME2)>(MA1-ME1)
     (data_teste['High'][i] - data_teste['Low'][i] ) > (data_teste['High'][i-1]  - data_teste['Low'][i-1] )
  ):
    #print(data_teste.index[i], 'sim')
    Eng_baixa.append('sim')
  else:
    #print(data_teste.index[i], 'nao')
    Eng_baixa.append('não')

def func_h_topo(i):
  if(
      #[PA1 < PF1]
     data_teste['Open'][i-1] <  data_teste['Close'][i-1]
     and 
     #[PF2 < PF1 ]
     data_teste['Close'][i] <  data_teste['Close'][i-1]
     and
     #[PA2 > PA1]
     data_teste['Open'][i]  > data_teste['Open'][i-1]
     and 
     #[(Pa2 - pf2)*3   <= (PF1 – PA1) >= (Pf2-Pa2) * 3]
     (
     ((data_teste['Close'][i-1] - data_teste['Open'][i-1]) / (data_teste['Open'][i]  - data_teste['Close'][i]) ) >= 3.0
     or 
     ((data_teste['Close'][i-1] - data_teste['Open'][i-1]) / (data_teste['Open'][i]  - data_teste['Close'][i]) ) <= -3.0
     )
  ):
    #print(data_teste.index[i], 'sim')
    Harami_topo.append('sim')
  else:
    Harami_topo.append('não')

def func_h_fundo(i):
  if(
      #[PA1 > PF1]
     data_teste['Open'][i-1] >  data_teste['Close'][i-1]
     and 
     #[PF2 > PF1 ]
     data_teste['Close'][i] >  data_teste['Close'][i-1]
     and
     #[PA2 < PA1]
     data_teste['Open'][i]  < data_teste['Open'][i-1]
     and 
     #[(Pa2 - pf2)*3   <= (PF1 – PA1) >= (Pf2-Pa2) * 3]
     (
     ((data_teste['Close'][i-1] - data_teste['Open'][i-1]) / (data_teste['Open'][i]  - data_teste['Close'][i]) ) >= 3.0
     or 
     ((data_teste['Close'][i-1] - data_teste['Open'][i-1]) / (data_teste['Open'][i]  - data_teste['Close'][i]) ) <= -3.0
     )
  ):
    #print(data_teste.index[i], 'sim')
    Harami_fundo.append('sim')
  else:
    Harami_fundo.append('não')


j_alta = []
J_baixa = []
Eng_alta = []
Eng_baixa = []
Harami_topo = []
Harami_fundo = []

#verificacao
for i in range(len(data_teste)):
  #janela de alta
  #print(i)
  func_j_alta(i)
  func_j_baixa(i)
  func_eng_alta(i)
  func_eng_baixa(i)
  func_h_topo(i)
  func_h_fundo(i)



#adiciona os resultados da verificacao ao df

data_teste['J_alta'] = j_alta
data_teste['J_baixa'] = J_baixa
data_teste['Eng_alta'] = Eng_alta
data_teste['Eng_baixa'] = Eng_baixa
data_teste['Harami_topo'] = Harami_topo
data_teste['Harami_fundo'] = Harami_fundo

#data_teste


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:104: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:106: RuntimeWarning: divide by zero encountered in double_scalars


In [ ]:
'''x = data_teste.index
y1 = data_teste['obv_teste']
y2 = data_teste['MMA_8_obvteste']
y3 = data_teste['MMA_20_obvteste']
plt.rcParams['figure.figsize'] = (20,7)
plt.plot(x, y1) 
plt.plot(x, y2)
plt.plot(x, y3)
plt.xlabel('x - axis') 
plt.ylabel('y1 - axis') 
plt.title('My first graph!') 
plt.show() '''

##

'''
x = data_teste.index
y2 = data_teste['OBV']
plt.plot(x, y2) 
plt.xlabel('x - axis') 
plt.ylabel('y2 - axis') 
plt.title('My first graph!') 
plt.show() '''

"\nx = data_teste.index\ny2 = data_teste['OBV']\nplt.plot(x, y2) \nplt.xlabel('x - axis') \nplt.ylabel('y2 - axis') \nplt.title('My first graph!') \nplt.show() "

In [ ]:
'''nao sei se quero usar esta forma de identificar a tendencia. eu disse na metodologia, que iria ver se a media 08 é maior ou menor que a media 20. entao é assim que devo fazer


tendencia = []

for i in range(len(data_teste)):
#for i in range(0,38):
  tendencia_alta = 1
  for j in range(6):
    if(data_teste['MME21'][i-1] < data_teste['MME21'][i-2]):
      tendencia_alta = 0
  #print(tendencia_alta)

  tendencia_baixa = 1
  for j in range(6):
    if(data_teste['MME21'][i-1] > data_teste['MME21'][i-2]):
      tendencia_baixa = 0

  if tendencia_alta == 1:
    tendencia.append('TA')
  elif tendencia_baixa == 1:
    tendencia.append('TB')
  else:
    tendencia.append('Lat')


data_teste['Tendencia'] = tendencia

'''

"nao sei se quero usar esta forma de identificar a tendencia. eu disse na metodologia, que iria ver se a media 08 é maior ou menor que a media 20. entao é assim que devo fazer\n\n\ntendencia = []\n\nfor i in range(len(data_teste)):\n#for i in range(0,38):\n  tendencia_alta = 1\n  for j in range(6):\n    if(data_teste['MME21'][i-1] < data_teste['MME21'][i-2]):\n      tendencia_alta = 0\n  #print(tendencia_alta)\n\n  tendencia_baixa = 1\n  for j in range(6):\n    if(data_teste['MME21'][i-1] > data_teste['MME21'][i-2]):\n      tendencia_baixa = 0\n\n  if tendencia_alta == 1:\n    tendencia.append('TA')\n  elif tendencia_baixa == 1:\n    tendencia.append('TB')\n  else:\n    tendencia.append('Lat')\n\n\ndata_teste['Tendencia'] = tendencia\n\n"

In [ ]:
#Verificando coeficiente angular

coef_ang = []
tend_teste = []
'''
NA METODOLOGIA EU DISSE QUE UTILIZARIA A MEDIA DE 20 PERIODOS PARA CALCULO DO COEF ANGULAR
for i in range(len(data_teste)):
  #x = (y2-y1) / (x2-x1)
  #y é  o preço e x é a data
  vary = data_teste['Close'][i] - data_teste['Close'][i-6]
  #varx = (data_teste.index[i]) - (data_teste.index[i-6])
  varx = 6 # 6 periodos
  x = vary / varx
  coef_ang.append(x)
  if  -0.4 < x < 0.4:
    tend_teste.append('TL')
  if x <= -0.4:
    tend_teste.append('TB')
  if x >= 0.4:
    tend_teste.append('TA')
'''

for i in range(len(data_teste)):
  #x = (y2-y1) / (x2-x1)
  #y é  o preço e x é a data
  vary = data_teste['MME20'][i] - data_teste['MME20'][i-6]
  #varx = (data_teste.index[i]) - (data_teste.index[i-6])
  varx = 6 # 6 periodos
  x = vary / varx
  coef_ang.append(x)
  if  -0.1 < x < 0.1:
    tend_teste.append('TL')
  elif x <= -0.1 and data_teste['MME8'][i] < data_teste['MME20'][i]:
    tend_teste.append('TB')
  elif x >= 0.1 and data_teste['MME8'][i] > data_teste['MME20'][i]:
    tend_teste.append('TA')
  else:
    tend_teste.append('NI')
  #print('dia, ', data_teste.index[i], 'vary ', vary, ' vary/6 ', x, ' medias 8 e 20: ', data_teste['MME8'][i], data_teste['MME20'][i] )
  
data_teste['Coef_ang'] = coef_ang
data_teste['tend_medias'] = tend_teste

#data_aux['Coef_ang'] = coef_ang
data_aux['tend_medias'] = tend_teste



In [ ]:
'''mask = (data_teste.index >  '2020-07-01') & (data_teste.index <= '2020-11-30')
filtered_df2=data_teste.loc[mask]
filtered_df2.iloc[:,6:]
#filtered_df2.iloc[:,22:]'''

"mask = (data_teste.index >  '2020-07-01') & (data_teste.index <= '2020-11-30')\nfiltered_df2=data_teste.loc[mask]\nfiltered_df2.iloc[:,6:]\n#filtered_df2.iloc[:,22:]"

In [ ]:
#utilizar proxima celula pois na proxima terá o ajuste da validação considerando topos e fundos
#Se sinal altista, verificar se está em tendencia de alta (através das médias), se Vol > MeanVol e se OBV > MeanOBV. estas serão as confirmações.

'''
tendencia = []
validacao = []

for i in range(len(data_teste)):
  tend = 'TL'
  #print(data_teste.index[i], data_teste["J_alta"][i] == "sim",  data_teste["Harami_fundo"][i], data_teste["Eng_alta"][i])
  verifica = "nao"
  #padroes altistas
  if(data_teste["J_alta"][i] == "sim" or data_teste["Harami_fundo"][i] == "sim" or data_teste["Eng_alta"][i] == "sim"):
    tend = 'TA'
    #print(data_teste.index[i], " padrao altista", 'mme9 ', data_teste["MME9"][i], 'mme21 ', data_teste["MME21"][i])
    if(data_teste["tend_teste"][i] == "TA"):
      #if(data_teste["MME8obv"][i] > data_teste["MME20obv"][i] ):
      if(data_teste["OBV"][i] > data_teste["MME20obv"][i] ):
        verifica = "Sim, TA"
        
  #padroes baixistas
  if(data_teste["J_baixa"][i] == "sim" or data_teste["Harami_topo"][i] == "sim" or data_teste["Eng_baixa"][i] == "sim"):
    tend = 'TB'
    #print(data_teste.index[i], " padrao altista", 'mme9 ', data_teste["MME9"][i], 'mme21 ', data_teste["MME21"][i])
    if(data_teste["tend_teste"][i] == "TB"):
      #print(data_teste.index[i], "Médias: Padrão altista com tendencia de Alta." )
      #if(data_teste["MME8obv"][i] < data_teste["MME20obv"][i] ):
      if(data_teste["OBV"][i] < data_teste["MME20obv"][i] ):
        verifica = "Sim, tB"
  validacao.append(verifica)
  tendencia.append(tend)

#print(validacao)
data_teste["sinal confirmado"] = validacao
data_teste["tendencia"] = tendencia

data_aux["tendencia"] = tendencia
data_aux["sinal confirmado"] = validacao


'''

'\ntendencia = []\nvalidacao = []\n\nfor i in range(len(data_teste)):\n  tend = \'TL\'\n  #print(data_teste.index[i], data_teste["J_alta"][i] == "sim",  data_teste["Harami_fundo"][i], data_teste["Eng_alta"][i])\n  verifica = "nao"\n  #padroes altistas\n  if(data_teste["J_alta"][i] == "sim" or data_teste["Harami_fundo"][i] == "sim" or data_teste["Eng_alta"][i] == "sim"):\n    tend = \'TA\'\n    #print(data_teste.index[i], " padrao altista", \'mme9 \', data_teste["MME9"][i], \'mme21 \', data_teste["MME21"][i])\n    if(data_teste["tend_teste"][i] == "TA"):\n      #if(data_teste["MME8obv"][i] > data_teste["MME20obv"][i] ):\n      if(data_teste["OBV"][i] > data_teste["MME20obv"][i] ):\n        verifica = "Sim, TA"\n        \n  #padroes baixistas\n  if(data_teste["J_baixa"][i] == "sim" or data_teste["Harami_topo"][i] == "sim" or data_teste["Eng_baixa"][i] == "sim"):\n    tend = \'TB\'\n    #print(data_teste.index[i], " padrao altista", \'mme9 \', data_teste["MME9"][i], \'mme21 \', data_teste

In [ ]:
#Se sinal altista, verificar se está em tendencia de alta (através das médias), se sentido dos topos do preços = sentido topos obv = tendencia
tendencia = []
validacao = []

for i in range(len(data_teste)):
  tend = ''
  #print(data_teste.index[i], data_teste["J_alta"][i] == "sim",  data_teste["Harami_fundo"][i], data_teste["Eng_alta"][i])
  verifica = ""
  #padroes altistas
  if(data_teste["J_alta"][i] == "sim" or data_teste["Harami_fundo"][i] == "sim" or data_teste["Eng_alta"][i] == "sim"):
    tend = 'TA'
    #print(data_teste.index[i], " padrao altista", 'mme9 ', data_teste["MME9"][i], 'mme21 ', data_teste["MME21"][i])
    if(data_teste['tend_medias'][i] == "TA" and data_teste['sentido_obv'][i] == data_teste['sentido_topos_preco'][i] == 'ASC'):
      verifica = "Sim, TA"
        
  #padroes baixistas
  if(data_teste["J_baixa"][i] == "sim" or data_teste["Harami_topo"][i] == "sim" or data_teste["Eng_baixa"][i] == "sim"):
    tend = 'TB'
    #print(data_teste.index[i], " padrao altista", 'mme9 ', data_teste["MME9"][i], 'mme21 ', data_teste["MME21"][i])
    if(data_teste['tend_medias'][i] == "TB" and data_teste['sentido_obv'][i] == data_teste['sentido_topos_preco'][i] == 'DESC'):
      verifica = "Sim, tB"

  validacao.append(verifica)
  tendencia.append(tend)

#print(validacao)
data_teste["tendencia_sinal"] = tendencia
data_teste["sinal confirmado"] = validacao


data_aux["tendencia_sinal"] = tendencia
data_aux["sinal confirmado"] = validacao

In [ ]:
#confirmacao do movimento
abertura = []
'''max = []
min = []'''

for i in range(len(data_teste)-1):
  abertura.append(data_teste['Open'][i+1])

for j in range(len(data_teste)-1,len(data_teste)):
  abertura.append(0)

data_teste['Conf_abert'] = abertura


#maior valor de fechamento dentro dos proximos 8periodos (B)

#porcentagem = B/A  * 100


In [ ]:
menor = []
maior = []


def min(i):
  lista=[]
  menor_aux = data_teste['Low'][i]
  for j in range(i+1,i+8):
    lista.append(data_teste['Low'][j])
    if data_teste['Low'][j] < menor_aux:
      menor_aux = data_teste['Low'][j]
  menor.append(menor_aux)

def max(i):
  lista=[]
  maior_aux = data_teste['High'][i]
  for j in range(i+1,i+8):
    lista.append(data_teste['High'][j])
    if data_teste['High'][j] > maior_aux:
      maior_aux = data_teste['High'][j]
  maior.append(maior_aux)


for i in range(len(data_teste)-8):
  min(i)
  max(i)

for i in range(len(data_teste)-8, len(data_teste)):
  menor.append(0)
  maior.append(0)


data_teste['conf_min'] = menor
data_teste['conf_max'] = maior

In [ ]:
percentual = []

for i in range(len(data_teste)):
  a = 0
  if data_teste['tend_medias'][i] == 'TB':
    a = 100 - (data_teste['conf_min'][i]  * 100 / data_teste['Conf_abert'][i] )
    #print('a', a)
  elif data_teste['tend_medias'][i] == 'TA':
    #print(' entrou em TA ', data_teste.index[i], data_teste['tend_medias'][i])
    a = (data_teste['conf_max'][i] * 100 / data_teste['Conf_abert'][i] ) - 100
  elif data_teste['tend_medias'][i] == 'TL':
    a = 270
  else:
    #print(' entrou no else ', data_teste.index[i], data_teste['tend_medias'][i])
    a = 250
  percentual.append(a)

data_teste['percentual'] = percentual
data_aux['percentual'] = percentual





In [ ]:
'''#df_mask=data_teste['Harami_fundo'] =='sim'
#df_mask=data_teste['Harami_topo'] =='sim'
#df_mask=data_teste['J_alta'] =='sim'
#df_mask=data_teste['J_baixa'] =='sim'
df_mask=data_teste['Eng_alta'] =='sim'
#df_mask=data_teste['Eng_baixa'] =='sim'
filtered_df = data_teste[df_mask]
filtered_df.iloc[:,6:]

#data_teste.iloc[:,6:]'''

"#df_mask=data_teste['Harami_fundo'] =='sim'\n#df_mask=data_teste['Harami_topo'] =='sim'\n#df_mask=data_teste['J_alta'] =='sim'\n#df_mask=data_teste['J_baixa'] =='sim'\ndf_mask=data_teste['Eng_alta'] =='sim'\n#df_mask=data_teste['Eng_baixa'] =='sim'\nfiltered_df = data_teste[df_mask]\nfiltered_df.iloc[:,6:]\n\n#data_teste.iloc[:,6:]"

In [ ]:
'''list(data_aux)'''

'list(data_aux)'

In [ ]:
mask = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste['J_baixa'] =='sim') & (data_teste['sinal confirmado'] != '')  & (data_teste['percentual'] > 1) & (data_teste['percentual'] < 250)
# & (data_teste['sinal confirmado'] != '') #& (data_teste['percentual'] > 1)

#(data_teste['Harami_fundo'] =='sim')
#data_teste['Harami_topo'] =='sim'
#data_teste['J_alta'] =='sim'
#data_teste['J_baixa'] =='sim'
#data_teste['Eng_alta'] =='sim'
#data_teste['Eng_baixa'] =='sim'
#(data_teste['tendencia_sinal'] == 'TA')
filtered_df = data_teste[mask]
filtered_df.iloc[: ,19:]#.shape[0]

,tend_medias,tendencia_sinal,sinal confirmado,Conf_abert,conf_min,conf_max,percentual
Date,,,,,,,
2020-03-06,TB,TB,"Sim, tB",17.809999,11.080000,24.129999,37.787758
2020-03-09,TB,TB,"Sim, tB",18.650000,10.870000,18.650000,41.715817
2020-03-12,TB,TB,"Sim, tB",15.100000,10.850000,15.400000,28.145695
2020-03-18,TB,TB,"Sim, tB",11.070000,10.850000,15.190000,1.987347
2020-09-25,TB,TB,"Sim, tB",20.330000,19.020000,20.700001,6.443677
2020-09-29,TB,TB,"Sim, tB",19.500000,19.020000,20.700001,2.461536
2020-10-07,TB,TB,"Sim, tB",20.040001,19.250000,20.590000,3.942120
2021-03-08,TB,TB,"Sim, tB",21.360001,20.860001,24.129999,2.340824
2021-09-08,TB,TB,"Sim, tB",25.080000,24.139999,26.459999,3.748009


In [ ]:
'''#com validacao do sinal
padrao = 'Harami_fundo'
resultados = []
mask0 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padrao] =='sim') # & (data_teste['sinal confirmado'] != '') #& (data_teste['percentual'] > 1)
#(data_teste['Harami_fundo'] =='sim')
#data_teste['Harami_topo'] =='sim'
#data_teste['J_alta'] =='sim'
#data_teste['J_baixa'] =='sim'
#data_teste['Eng_alta'] =='sim'
#data_teste['Eng_baixa'] =='sim'
#(data_teste['tendencia_sinal'] == 'TA')
filtered_df = data_teste[mask0]
resultados.append(filtered_df.iloc[: ,19:].shape[0])

mask1 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padrao] =='sim')  & (data_teste['sinal confirmado'] != '') #& (data_teste['percentual'] > 1)
filtered_df = data_teste[mask1]
resultados.append(filtered_df.iloc[: ,19:].shape[0])

mask2 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padrao] =='sim')  & (data_teste['sinal confirmado'] != '') & (data_teste['percentual'] > 1)
filtered_df = data_teste[mask2]
resultados.append(filtered_df.iloc[: ,19:].shape[0])

resultados.append(resultados[2] *100/resultados[1])

resultados'''

"#com validacao do sinal\npadrao = 'Harami_fundo'\nresultados = []\nmask0 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padrao] =='sim') # & (data_teste['sinal confirmado'] != '') #& (data_teste['percentual'] > 1)\n#(data_teste['Harami_fundo'] =='sim')\n#data_teste['Harami_topo'] =='sim'\n#data_teste['J_alta'] =='sim'\n#data_teste['J_baixa'] =='sim'\n#data_teste['Eng_alta'] =='sim'\n#data_teste['Eng_baixa'] =='sim'\n#(data_teste['tendencia_sinal'] == 'TA')\nfiltered_df = data_teste[mask0]\nresultados.append(filtered_df.iloc[: ,19:].shape[0])\n\nmask1 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padrao] =='sim')  & (data_teste['sinal confirmado'] != '') #& (data_teste['percentual'] > 1)\nfiltered_df = data_teste[mask1]\nresultados.append(filtered_df.iloc[: ,19:].shape[0])\n\nmask2 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padrao] =='sim')  & (data_teste['

In [ ]:
'''#sem validacao do sinal
padrao = 'Harami_fundo'
resultados = []
mask0 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padrao] =='sim') # & (data_teste['sinal confirmado'] != '') #& (data_teste['percentual'] > 1)
filtered_df = data_teste[mask0]
resultados.append(filtered_df.iloc[: ,19:].shape[0])

mask2 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padrao] =='sim')  & (data_teste['percentual'] > 1) & (data_teste['percentual'] < 250)
filtered_df = data_teste[mask2]
resultados.append(filtered_df.iloc[: ,19:].shape[0])

resultados.append(resultados[1] *100/resultados[0])
resultados'''

"#sem validacao do sinal\npadrao = 'Harami_fundo'\nresultados = []\nmask0 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padrao] =='sim') # & (data_teste['sinal confirmado'] != '') #& (data_teste['percentual'] > 1)\nfiltered_df = data_teste[mask0]\nresultados.append(filtered_df.iloc[: ,19:].shape[0])\n\nmask2 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padrao] =='sim')  & (data_teste['percentual'] > 1) & (data_teste['percentual'] < 250)\nfiltered_df = data_teste[mask2]\nresultados.append(filtered_df.iloc[: ,19:].shape[0])\n\nresultados.append(resultados[1] *100/resultados[0])\nresultados"

In [ ]:
#COM validacao do sinal
padroes = ['Harami_fundo',  'Harami_topo', 'J_alta', 'J_baixa', 'Eng_alta', 'Eng_baixa']
resultados = []
for i in range(len(padroes)):
  resultados = [padroes[i]]
  #FP
  mask0 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31')& ( data_teste[padroes[i]] =='sim')
  filtered_df = data_teste[mask0]
  resultados.append(filtered_df.iloc[: ,19:].shape[0])

  #VP
  mask1 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padroes[i]] =='sim')  & (data_teste['sinal confirmado'] != '') 
  filtered_df = data_teste[mask1]
  resultados.append(filtered_df.iloc[: ,19:].shape[0])

  #AM
  mask2 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padroes[i]] =='sim')  & (data_teste['sinal confirmado'] != '') & (data_teste['percentual'] > 1) & (data_teste['percentual'] < 250)
  filtered_df = data_teste[mask2]
  resultados.append(filtered_df.iloc[: ,19:].shape[0])

  #EF
  try: 
    resultados[3] *100/resultados[2]
    resultados.append(resultados[3] *100/resultados[2])
  except:
    resultados.append(0)

  print(resultados)

['Harami_fundo', 24, 1, 1, 100.0]
['Harami_topo', 22, 1, 1, 100.0]
['J_alta', 8, 3, 3, 100.0]
['J_baixa', 86, 9, 9, 100.0]
['Eng_alta', 12, 2, 2, 100.0]
['Eng_baixa', 7, 0, 0, 0]


In [ ]:
#sem validacao do sinal
padroes = ['Harami_fundo',  'Harami_topo', 'J_alta', 'J_baixa', 'Eng_alta', 'Eng_baixa']
resultados = []
for i in range(len(padroes)):
  resultados = [padroes[i]]
  #FP
  mask0 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31')& ( data_teste[padroes[i]] =='sim')
  filtered_df = data_teste[mask0]
  resultados.append(filtered_df.iloc[: ,19:].shape[0])
  
  #AM
  mask2 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padroes[i]] =='sim')  & (data_teste['percentual'] > 1) & (data_teste['percentual'] < 250)
  filtered_df = data_teste[mask2]
  resultados.append(filtered_df.iloc[: ,19:].shape[0])

  #EF
  resultados.append(resultados[2] *100/resultados[1])

  print(resultados)

['Harami_fundo', 24, 8, 33.333333333333336]
['Harami_topo', 22, 5, 22.727272727272727]
['J_alta', 8, 5, 62.5]
['J_baixa', 86, 25, 29.069767441860463]
['Eng_alta', 12, 3, 25.0]
['Eng_baixa', 7, 2, 28.571428571428573]


In [ ]:
'''#sem validacao do sinal
padroes = ['Harami_fundo',  'Harami_topo', 'J_alta', 'J_baixa', 'Eng_alta', 'Eng_baixa']
resultados = []
for i in range(len(padroes)):

  print([padroes[i]])
  #FP
  mask0 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31')& ( data_teste[padroes[i]] =='sim') & (data_teste['sinal confirmado']== '') & (data_teste['tend_medias']!= 'NI')
  print(filtered_df.iloc[: ,19:])
'''


"#sem validacao do sinal\npadroes = ['Harami_fundo',  'Harami_topo', 'J_alta', 'J_baixa', 'Eng_alta', 'Eng_baixa']\nresultados = []\nfor i in range(len(padroes)):\n\n  print([padroes[i]])\n  #FP\n  mask0 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31')& ( data_teste[padroes[i]] =='sim') & (data_teste['sinal confirmado']== '') & (data_teste['tend_medias']!= 'NI')\n  print(filtered_df.iloc[: ,19:])\n"